In [36]:
%pip install requests


[notice] A new release of pip available: 22.2.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [37]:
import pandas as pd
import numpy as np
import requests

In [38]:
views = pd.read_json('../ex02/auto.json')
#views = views.set_index('CarNumber')
pd.options.display.float_format = '{:,.2f}'.format 
print(views)

        CarNumber  Refund     Fines    Make    Model
0    Y163O8161RUS       2  3,200.00    Ford    Focus
1     E432XX77RUS       1  6,500.00  Toyota    Camry
2     7184TT36RUS       1  2,100.00    Ford    Focus
3    X582HE161RUS       2  2,000.00    Ford    Focus
4    92918M178RUS       1  5,700.00    Ford    Focus
..            ...     ...       ...     ...      ...
720  Y163O8161RUS       2  1,600.00    Ford    Focus
721  M0309X197RUS       1 22,300.00    Ford    Focus
722  O673E8197RUS       2    600.00    Ford    Focus
723  8610T8154RUS       1  2,000.00    Ford    Focus
724  H419XE197RUS       2  8,594.59  Toyota  Corolla

[725 rows x 5 columns]


#### Enrich the dataframe using a sample from that dataframe

In [39]:
np.random.seed(21)
sampled_combinations = views[['CarNumber', 'Make', 'Model']].drop_duplicates().sample(n=200, replace=True, random_state=21)

sampled_data = sampled_combinations.assign(
    Fines=np.random.choice(views['Fines'], size=200),      # Reuse existing fine values
    Refund=np.random.choice(views['Refund'], size=200)     # Reuse existing refund values
)
sampled_data = sampled_data.reset_index(drop=True)
print(sampled_data)
concat_rows = pd.concat([views, sampled_data])
concat_rows = concat_rows.set_index('CarNumber')
print(concat_rows)


        CarNumber        Make  Model     Fines  Refund
0     T6329O50RUS        Ford  Focus  1,500.00       1
1     H917TC36RUS        Ford  Focus  4,000.00       2
2    9763HY161RUS        Ford  Focus  4,500.00       2
3    H003MH197RUS        Ford  Focus  2,000.00       2
4    O64097197RUS        Ford  Focus  1,300.00       2
..            ...         ...    ...       ...     ...
195   X782CO96RUS        Ford  Focus  2,000.00       1
196  E79988152RUS        Ford  Focus    400.00       2
197  X582HE161RUS        Ford  Focus 12,800.00       1
198  9594HY161RUS        Ford  Focus    800.00       2
199  K327T8197RUS  Volkswagen   Golf  2,200.00       1

[200 rows x 5 columns]
              Refund     Fines        Make  Model
CarNumber                                        
Y163O8161RUS       2  3,200.00        Ford  Focus
E432XX77RUS        1  6,500.00      Toyota  Camry
7184TT36RUS        1  2,100.00        Ford  Focus
X582HE161RUS       2  2,000.00        Ford  Focus
92918M178RUS    

#### Enrich the dataframe concat_rows by a new column with the data generated

In [40]:
np.random.seed(21)
Years = pd.Series(
    np.random.randint(1980,2020, size=200), 
    name='Year'
)
fines = pd.concat([sampled_data, Years], axis=1)
if 'CarNumber' in fines.columns:
    fines.set_index('CarNumber', inplace=True)
elif 'CarNumber' != fines.index.name:
    print('Warning: "CarNumber" not found in columns or index')

print(fines)

                    Make  Model     Fines  Refund  Year
CarNumber                                              
T6329O50RUS         Ford  Focus  1,500.00       1  1989
H917TC36RUS         Ford  Focus  4,000.00       2  1995
9763HY161RUS        Ford  Focus  4,500.00       2  1984
H003MH197RUS        Ford  Focus  2,000.00       2  2015
O64097197RUS        Ford  Focus  1,300.00       2  2014
...                  ...    ...       ...     ...   ...
X782CO96RUS         Ford  Focus  2,000.00       1  1991
E79988152RUS        Ford  Focus    400.00       2  1995
X582HE161RUS        Ford  Focus 12,800.00       1  1993
9594HY161RUS        Ford  Focus    800.00       2  2015
K327T8197RUS  Volkswagen   Golf  2,200.00       1  1999

[200 rows x 5 columns]


##### Enrich the dataframe with the data from another dataframe

##### Create a new series with the surnames (they should not have special characters like commas, brackets, etc.) from the data you gathered, the count should be equal to the number of unique car numbers using the sample (use random_state = 21)

In [41]:
surnames = pd.read_json('../data/surname.json')
surnames.columns = surnames.iloc[0]
surnames = surnames[1:]
surnames = surnames['NAME'] 
np.random.seed(21)
random_surnames = pd.Series(
    np.random.choice(surnames, size=concat_rows.index.nunique(), replace=True), 
    name='Surname'
)
print(random_surnames)

0      RICHARDSON
1            ROSS
2          MORGAN
3          BAILEY
4           LOPEZ
          ...    
526      CAMPBELL
527          HALL
528         BAKER
529          DIAZ
530        MORGAN
Name: Surname, Length: 531, dtype: object


##### Create the dataframe owners with 2 columns: CarNumber and SURNAME

In [42]:
owners = pd.Series(
    random_surnames.values,  
    name='Owner',
    index=concat_rows.index.unique()
)
print(owners)


CarNumber
Y163O8161RUS    RICHARDSON
E432XX77RUS           ROSS
7184TT36RUS         MORGAN
X582HE161RUS        BAILEY
92918M178RUS         LOPEZ
                   ...    
O136HO197RUS      CAMPBELL
O22097197RUS          HALL
M0309X197RUS         BAKER
O673E8197RUS          DIAZ
8610T8154RUS        MORGAN
Name: Owner, Length: 531, dtype: object


##### Append 5 more observations to the fines dataframe (come up with your own ideas of CarNumber, etc.)

In [43]:
new_fines = pd.DataFrame(
    {'CarNumber':['A111AA111RUS', 'B222BB222RUS', 'C333CC333RUS', 'D444DD444RUS', 'E555EE555RUS'],
    'Make':['Ford', 'Toyota', 'Volkswagen', 'Volkswagen', 'Ford'],
    'Model':['Focus', 'Corolla', 'Golf', 'Passat', 'Mondeo'],
    'Fines':[2500.00, 1500.00, 3000.00, 2000.00, 3500.00],
    'Refund':[1, 1, 2, 1, 1],
    'Year':[2015, 2017, 2000, 2013, 1999]}
)
new_fines = new_fines.set_index('CarNumber')
fines = pd.concat([fines, new_fines])
print(fines)

                    Make    Model    Fines  Refund  Year
CarNumber                                               
T6329O50RUS         Ford    Focus 1,500.00       1  1989
H917TC36RUS         Ford    Focus 4,000.00       2  1995
9763HY161RUS        Ford    Focus 4,500.00       2  1984
H003MH197RUS        Ford    Focus 2,000.00       2  2015
O64097197RUS        Ford    Focus 1,300.00       2  2014
...                  ...      ...      ...     ...   ...
A111AA111RUS        Ford    Focus 2,500.00       1  2015
B222BB222RUS      Toyota  Corolla 1,500.00       1  2017
C333CC333RUS  Volkswagen     Golf 3,000.00       2  2000
D444DD444RUS  Volkswagen   Passat 2,000.00       1  2013
E555EE555RUS        Ford   Mondeo 3,500.00       1  1999

[205 rows x 5 columns]


##### Delete the dataframe last 20 observations from the owners and add 3 new observations (they are not the same as those you add to the fines dataframe)

In [44]:
owners = owners.iloc[:-20]
owners = pd.concat([owners, pd.Series(['ROSSK', 'KOMMO', 'LEN'], index=['A123AA123RUS', 'B234BB234RUS', 'C345CC345RUS'],  name=owners.name )])
print(owners)

Y163O8161RUS    RICHARDSON
E432XX77RUS           ROSS
7184TT36RUS         MORGAN
X582HE161RUS        BAILEY
92918M178RUS         LOPEZ
                   ...    
O50197197RUS        WRIGHT
7608EE777RUS          HILL
A123AA123RUS         ROSSK
B234BB234RUS         KOMMO
C345CC345RUS           LEN
Name: Owner, Length: 514, dtype: object


##### Join both dataframes

###### The new dataframe should have only the car numbers that exist in both dataframes

In [45]:
merged_inner = fines.join(owners, how='inner')
print(merged_inner)

                    Make  Model     Fines  Refund  Year      Owner
CarNumber                                                         
T6329O50RUS         Ford  Focus  1,500.00       1  1989    SANDERS
H917TC36RUS         Ford  Focus  4,000.00       2  1995   ROBINSON
9763HY161RUS        Ford  Focus  4,500.00       2  1984     TORRES
H003MH197RUS        Ford  Focus  2,000.00       2  2015       WARD
O64097197RUS        Ford  Focus  1,300.00       2  2014  HERNANDEZ
...                  ...    ...       ...     ...   ...        ...
X782CO96RUS         Ford  Focus  2,000.00       1  1991   CASTILLO
E79988152RUS        Ford  Focus    400.00       2  1995      ALLEN
X582HE161RUS        Ford  Focus 12,800.00       1  1993     BAILEY
9594HY161RUS        Ford  Focus    800.00       2  2015    BENNETT
K327T8197RUS  Volkswagen   Golf  2,200.00       1  1999     TURNER

[192 rows x 6 columns]


###### The new dataframe should have all the car numbers that exist in both dataframes

In [46]:
merged_outer = fines.join(owners, how='outer')
print(merged_outer)

              Make  Model  Fines    Refund     Year     Owner
CarNumber                                                    
704687163RUS   NaN    NaN    NaN       NaN      NaN     ADAMS
704787163RUS   NaN    NaN    NaN       NaN      NaN    MORGAN
704987163RUS   NaN    NaN    NaN       NaN      NaN  MITCHELL
705287163RUS   NaN    NaN    NaN       NaN      NaN     GOMEZ
705387163RUS  Ford  Focus 300.00 10,700.00 1,995.00   STEWART
...            ...    ...    ...       ...      ...       ...
Y965O8197RUS   NaN    NaN    NaN       NaN      NaN  PETERSON
Y966O8197RUS   NaN    NaN    NaN       NaN      NaN     WHITE
Y967O8197RUS  Ford  Focus 500.00      1.00 1,983.00    MARTIN
Y969O8197RUS   NaN    NaN    NaN       NaN      NaN     LOPEZ
Y973O8197RUS   NaN    NaN    NaN       NaN      NaN     YOUNG

[561 rows x 6 columns]


###### The new dataframe should have only the car numbers from the fines dataframe

In [47]:
merged_left = fines.join(owners, how='left')
print(merged_left)

                    Make    Model    Fines  Refund  Year      Owner
CarNumber                                                          
T6329O50RUS         Ford    Focus 1,500.00       1  1989    SANDERS
H917TC36RUS         Ford    Focus 4,000.00       2  1995   ROBINSON
9763HY161RUS        Ford    Focus 4,500.00       2  1984     TORRES
H003MH197RUS        Ford    Focus 2,000.00       2  2015       WARD
O64097197RUS        Ford    Focus 1,300.00       2  2014  HERNANDEZ
...                  ...      ...      ...     ...   ...        ...
A111AA111RUS        Ford    Focus 2,500.00       1  2015        NaN
B222BB222RUS      Toyota  Corolla 1,500.00       1  2017        NaN
C333CC333RUS  Volkswagen     Golf 3,000.00       2  2000        NaN
D444DD444RUS  Volkswagen   Passat 2,000.00       1  2013        NaN
E555EE555RUS        Ford   Mondeo 3,500.00       1  1999        NaN

[205 rows x 6 columns]


###### The new dataframe should have only the car numbers from the owners dataframe

In [48]:
merged_right = fines.join(owners, how='right')
print(merged_right)

                Make    Model     Fines  Refund     Year       Owner
CarNumber                                                           
Y163O8161RUS    Ford    Focus 27,000.00    2.00 1,990.00  RICHARDSON
Y163O8161RUS    Ford    Focus    800.00    2.00 1,994.00  RICHARDSON
E432XX77RUS   Toyota    Camry  1,000.00    2.00 2,015.00        ROSS
E432XX77RUS   Toyota    Camry  3,800.00    2.00 1,984.00        ROSS
7184TT36RUS      NaN      NaN       NaN     NaN      NaN      MORGAN
...              ...      ...       ...     ...      ...         ...
O50197197RUS    Ford    Focus  6,000.00    2.00 2,011.00      WRIGHT
7608EE777RUS   Skoda  Octavia    500.00    1.00 1,990.00        HILL
A123AA123RUS     NaN      NaN       NaN     NaN      NaN       ROSSK
B234BB234RUS     NaN      NaN       NaN     NaN      NaN       KOMMO
C345CC345RUS     NaN      NaN       NaN     NaN      NaN         LEN

[548 rows x 6 columns]


##### Create a pivot table from the fines dataframe, it should look like this (the values are the sums of the fines), but with all the years (the values may be different for you):

In [49]:
pivot_table = pd.pivot_table(
    fines,
    values='Fines',
    index=['Make', 'Model'], 
    columns='Year',
    aggfunc='sum'
)
print(pivot_table)

Year                    1980      1981      1982      1983      1984  \
Make       Model                                                       
Ford       Focus   23,494.59 33,000.00 17,300.00 18,100.00 14,400.00   
           Mondeo        NaN 18,800.00       NaN       NaN       NaN   
Skoda      Octavia       NaN  7,600.00       NaN       NaN       NaN   
Toyota     Camry         NaN       NaN       NaN       NaN 11,300.00   
           Corolla       NaN       NaN       NaN       NaN       NaN   
Volkswagen Golf          NaN       NaN       NaN  6,800.00       NaN   
           Jetta         NaN       NaN       NaN       NaN       NaN   
           Passat        NaN       NaN       NaN       NaN       NaN   
           Touareg       NaN       NaN       NaN       NaN       NaN   

Year                    1985     1986      1987      1988      1989  ...  \
Make       Model                                                     ...   
Ford       Focus   53,794.59      NaN 11,900.00 15,100.

##### Save both the fines and owners dataframes to CSV files 

In [50]:
fines.to_csv('fines.csv')
owners.to_csv('owners.csv')

In [51]:
concat_rows.count()
len(concat_rows)

925